In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
import numpy as np
import jack
from lmz import *

In [ ]:
import scanpy as sc
import cellsaw as cs

In [ ]:
!echo $PYTHONPATH

In [ ]:
import lucy.io as io
#io.load_timeseries()
io.load_scib()

# LOAD DATA

In [ ]:
# zedata = [sc.read('/home/ubuntu/benchdata/'+data) for data in "Immune_ALL_hum_mou.h5ad  Immune_ALL_human.h5ad  Lung_atlas_public.h5ad  human_pancreas_norm_complexBatch.h5ad".split()]
# batch,typ = Transpose (Map(lambda x:x.split(), 'batch final_annotation#batch final_annotation#batch cell_type#tech celltype'.split("#")))

# zedata =  [[z[z.obs[b]==i] for i in z.obs[b].unique()] for z,b in zip(zedata, batch)]

datasets = io.load_scib() + io.load_timeseries()


def samplecopy(data,num, seed):
    np.random.seed(seed) 
    obs_indices = np.random.choice(data.n_obs, size=num, replace=True) 
    r =  data[obs_indices].copy()
    r.obs_names_make_unique()
    return r

ssdata = [[samplecopy(i,750,31442)  for i in series[:20]]  for series in datasets] 

[cs.preprocess.annotate_genescore2(data,selector = 'cell_ranger', normrow = True, log = True) 
 for data in ssdata]
[[ sc.pp.normalize_total(d,1e4) for d in data] for data in ssdata]
[[sc.pp.log1p(d) for d in data] for data  in ssdata]
pass

# optimize integration

In [ ]:
from sklearn import neighbors as nbrs


def neighborgraph(x, neighbors):
    z= nbrs.kneighbors_graph(x,neighbors)
    diff = z-z.T
    diff[diff > 0 ] = 0
    z-= diff
    return z

def nnn(x, neighbors,near = True):
    z = np.zeros_like(x)
    np.fill_diagonal(x,np.NINF)
    for i,row in enumerate(x):
        sr = np.argsort(row)
        z[i][sr[-neighbors:]]  = 1
    diff = z-z.T
    diff[diff > 0 ] = 0
    z-= diff
    return z


nnn(merge.similarity,1)

In [ ]:
nnn(-merge.similarity,1)

In [ ]:
from sklearn.metrics import  silhouette_score
import ubergauss.tools as ug
from natto.out.quality import clust as nnpurity 
from cellsaw.merge.diffusion import kernel
from cellsaw import merge as merg
import cellsaw as cs
import mnnpy
import umap

from hyperopt import fmin, tpe, hp, Trials
from hyperopt.pyll import scope
    
def evalp(  x =None,neighs=5,conns=1, scal=1,comp=5,umaps = 0,outl = .75, 
          scalecon = .25, 
          pca = 40, connect = 100,nalg = 0):
    
    batch_,label = 'batch', 'label'
    data = [z.copy() for z in ssdata[x]]
    if umaps == 0:
        merge = merg.Merge(data,umaps=[],oldcut=False, pca = pca)
        data = merge.projections[1]
    else:
        merge = merg.Merge(data,umaps=[umaps],oldcut=False, pca  = pca)
        data = merge.projections[2]
    dist = kernel.linear_assignment_integrate(data,
                                              intra_neigh = neighs, 
                                              inter_neigh = conns, 
                                              scaling_num_neighbors = scal,
                                              outlier_threshold = outl,
                                              scaling_threshold = scalecon,
                                              dataset_similarity = (merge.similarity, connect, nalg))
    sim = kernel.distmatrixumap(data,dist,components = comp)
    y = [ x for a in merge.data for x in a.obs[label] ]
    score = nnpurity(sim,y)+2*silhouette_score(sim,y)
    return score


space3 = { 
    'neighs' : scope.int(hp.quniform('neighs',3,25,1)),
    'conns' : scope.int(hp.quniform('conns',1,5,1)),
    'scal' : scope.int(hp.quniform('scal',1,5,1)),
    'comp' : scope.int(hp.quniform('comp',4,30,1)),
    'umaps' : scope.int(hp.quniform('umaps',0,20,5)),
    'scalecon' : hp.uniform('scalecon',.05,.95),
    'outl' : hp.uniform('outl',.5,.95),
    'nalg' : hp.choice ('nalg',[True, False]),
    'pca' : scope.int(hp.quniform('pca',30,50,1)),
    'connect' : scope.int(hp.quniform('connect',10,100,1))
}

def eva(x):
    space3['x'] =  hp.choice ('x',[x]),
    trials3 = Trials()
    best = fmin(fn=evalp, algo=tpe.suggest,
                trials = trials3,
                space = space3, 
                max_evals=40) 
    print(best)
    return trials3, best

re = ug.xmap(eva, Range(ssdata))


In [ ]:
asd = Map(len, ssdata)
for d in re:
    d['connect2']= d['connect'] * asd[d['x']]

In [ ]:
import pandas as pd
import seaborn as sns
data = pd.DataFrame(re)
interest = 'connect'
sns.boxplot(data = data, x = interest, y= 'score', hue = 'nalg')
plt.show()
sns.scatterplot(data = data, x = interest, y= 'score', hue = 'x')
plt.show() # was around .9 at best withthe other neighbor function..

In [ ]:
from sklearn.metrics import  silhouette_score
import ubergauss.tools as ug
from natto.out.quality import clust as nnpurity 
from cellsaw.merge.diffusion import kernel
from cellsaw import merge as merg
import cellsaw as cs
import mnnpy
import umap
from sklearn import manifold as mf


# first we get the integration data:
def evalp(x,neighs=5,conns=1, scal=1,comp=5,umaps = 0,outl = .75, scalecon = .25):
    batch_,label = batch[x], typ[x]
    data = [z.copy() for z in ssdata[x]]
    if umaps == 0:
        merge = merg.Merge(data,umaps=[],oldcut=False, pca = 40)
        data = merge.projections[1]
    else:
        merge = merg.Merge(data,umaps=[umaps],oldcut=False, pca  = 40)
        data = merge.projections[2]
    dist = kernel.linear_assignment_integrate(data,
                                              intra_neigh = neighs, 
                                              inter_neigh = conns, 
                                              scaling_num_neighbors = scal,
                                               outlier_threshold = outl,
                                               scaling_threshold = scalecon)
    y = [ x for a in merge.data for x in a.obs[label] ]
    return y, dist, data

params =  { 'conns': 1, 'neighs': 16, 'outl': 0.8079195869285946, 'scal': 2, 
           'scalecon': 0.9365341951145691, 'umaps': 0}
data = ug.xmap(lambda x:evalp(x,**params),Range(ssdata))

# then we set up the new optimization:


In [ ]:

import time
methods = []

def umap(dist, comp, data):
    return kernel.distmatrixumap(data,dist,components = 5)
methods.append(umap)

'''
manifold.Isomap(*[, n_neighbors, radius, ...])
manifold.LocallyLinearEmbedding(*[, ...]) NO PRECOMP
manifold.MDS([n_components, metric, n_init, ...])
manifold.SpectralEmbedding([n_components, ...])
manifold.TSNE([n_components, perplexity, ...])
manifold.locally_linear_embedding(X, *, ...)
manifold.spectral_embedding(adjacency, *[, ...])
manifold.trustworthiness(
'''

def isomap(dist, comp, data):
    iso = mf.Isomap(n_neighbors = comp,n_components=5, metric = 'precomputed' )
    return iso.fit_transform(dist)#kernel.distmatrixumap(data,dist,components = comp)
methods.append(isomap)

def spectral(dist, comp, data):
    return mf.SpectralEmbedding(n_components = 5,n_neighbors = comp,affinity = 'precomputed').fit_transform(dist)#data,dist,components = comp)
methods.append(spectral)

def MDS(dist, comp, data):
    return mf.MDS(n_components = 5, metric =False, dissimilarity = "precomputed").fit_transform(dist.todense())# ,dist,components = comp)
methods.append(MDS)

def TSNE(dist, comp, data):
    return mf.TSNE(n_components = 5,early_exaggeration = comp,method='exact', metric = "precomputed").fit_transform(dist.todense())
methods.append(TSNE)

def evalpp(arg):
    x,method,val = arg
    s = time.time()
    y,dist, pca = data[x]
    # sim = methods[method](dist, val, pca)
    try:
        sim = methods[method](dist, val, pca)
    except:
        return 0,0,0,0,0
    
    return nnpurity(sim,y)+2*silhouette_score(sim,y),time.time()-s, x, method, val

'''
def runall(params):
    return -np.mean(ug.xmap(lambda x:evalpp(x,**params),Range(data)))
    
import ubergauss.hyperopt as uo
space = [f'method 0 {len(methods)-1} 1','val 5 20 1']
ship = uo.spaceship(space)
trials = uo.Trials()
print("start")
best = uo.fmin(fn=runall, algo=uo.tpe.suggest,
            trials = trials,
            space = ship.space, 
            max_evals=40) 

'''




runs = [(ds,meth,val) for ds in Range(data) for meth in Range(methods) for val in range(5,20)]
# evalpp(runs[0])
r = ug.xmap(evalpp,runs )





In [ ]:
import pandas as pd
z = pd.DataFrame(r, columns='score time ds method val'.split())

In [ ]:
import seaborn as sns
sns.pairplot(z, hue = 'ds')